<a href="https://colab.research.google.com/github/justmeanmol/Mnist/blob/main/Keyword_Extraction_From_Resume_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Mini Project: Automatic Keyword Extraction using Sentence Transformers

##  Objective
In this mini-project, we’ll build a simple **semantic keyword extractor** using the **SentenceTransformer** model.  
Given any input text, the model will find the **most relevant keywords** (topics) based on **cosine similarity** between the text and a predefined list of keywords.


In [ ]:
# !pip install torch
# !pip install sentence_transformers
# !pip install tf-keras


In [1]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

- all-MiniLM-L6-v2 is a pre-trained Sentence Transformer model used to convert text into meaningful numerical vectors (embeddings).
- It understands the meaning of sentences and converts them into numbers.

In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 🔹Cosine Similarity

Cosine similarity measures **how close two embeddings are**.  
It’s widely used to check **semantic similarity** between two pieces of text.


In [3]:
sentences = ["I love playing cricket", "i am fine."]
# Generate embeddings
embeddings = model.encode(sentences)

print("Sentence Embeddings Shape:", embeddings.shape)
# embeddings[0]

Sentence Embeddings Shape: (2, 384)


In [4]:
# embeddings[1]

In [5]:
# Calculate cosine similarity between the two sentences
similarity_score = util.cos_sim(embeddings[0], embeddings[1])

print(f"Cosine Similarity: {similarity_score.item():.4f}")

Cosine Similarity: 0.1219


## Define Domain Keywords
We’ll create a list of keywords covering multiple domains such as **AI, sports, science, politics, health**, etc.


In [7]:
# Expanded Keywords
domain_keywords = {

    "Technology": [
        "artificial intelligence", "machine learning", "deep learning",
        "neural network", "nlp", "computer vision", "chatgpt",
        "algorithm", "data science", "big data", "blockchain",
        "cybersecurity", "cloud computing", "software engineering",
        "programming", "coding", "python", "java", "api",
        "database", "sql", "devops", "automation", "internet of things",
        "iot", "web development", "mobile app"
    ],

    "Sports": [
        "football", "soccer", "cricket", "basketball", "tennis",
        "badminton", "athletics", "olympics", "world cup",
        "tournament", "match", "player", "coach", "team",
        "goal", "score", "stadium", "league", "fitness",
        "training", "workout", "medal", "championship"
    ],

    "Politics": [
        "election", "vote", "voting", "government", "parliament",
        "president", "prime minister", "minister", "policy",
        "legislation", "law", "constitution", "democracy",
        "political party", "campaign", "diplomacy",
        "foreign relations", "international affairs", "governance"
    ],

    "Science": [
        "biology", "physics", "chemistry", "genetics",
        "astronomy", "astrophysics", "space", "nasa",
        "experiment", "research", "laboratory", "scientist",
        "theory", "discovery", "quantum", "evolution",
        "climate science", "scientific study"
    ],

    "Health": [
        "health", "healthcare", "medicine", "medical",
        "doctor", "hospital", "patient", "disease",
        "treatment", "therapy", "vaccine", "diagnosis",
        "mental health", "psychology", "nutrition",
        "diet", "fitness", "exercise", "wellness",
        "public health"
    ],

    "Business & Economics": [
        "business", "startup", "entrepreneurship",
        "market", "marketing", "finance", "economics",
        "investment", "stock", "share", "trading",
        "revenue", "profit", "loss", "budget",
        "inflation", "economic growth", "supply chain",
        "demand", "industry"
    ],

    "Environment": [
        "environment", "climate change", "global warming",
        "pollution", "air pollution", "water pollution",
        "sustainability", "renewable energy", "solar energy",
        "wind energy", "conservation", "ecosystem",
        "biodiversity", "deforestation", "carbon emissions",
        "greenhouse gases"
    ],

    "Entertainment": [
        "movie", "film", "cinema", "actor", "actress",
        "music", "song", "album", "concert",
        "bollywood", "hollywood", "web series",
        "television", "tv show", "director",
        "trailer", "box office", "streaming"
    ],

    "Education": [
        "education", "school", "college", "university",
        "student", "teacher", "professor", "lecture",
        "exam", "test", "assignment", "syllabus",
        "curriculum", "degree", "online learning",
        "e-learning", "training", "skill development"
    ],

    "Psychology & Social": [
        "psychology", "behavior", "mental health",
        "emotion", "stress", "anxiety", "depression",
        "cognitive", "personality", "therapy",
        "social behavior", "society", "culture",
        "human behavior"
    ]
}


##  Building the Keyword Extraction Function
We’ll define a function that encodes the input text and compares it with each keyword using **cosine similarity**.  
The top few matches (highest similarity scores) will be selected as the most relevant keywords.


In [8]:
# Function to extract top N matching keywords

def extract_top_keywords(content,keywords,keyword_embeddings,top_n=5):
    keyword_embeddings = model.encode(keywords)
    # Encode the content
    content_embeddings =model.encode(content)

    # Compute cosine similarities
    similarites = util.cos_sim(content_embeddings,keyword_embeddings).flatten()

    # Get indices of top N similarities
    top_indices = torch.topk(similarites,top_n).indices

    # Retrieve top N keywords based on the indices
    top_keywords = [keywords[i] for i in top_indices]

    return top_keywords



##  Running the Interactive Keyword Extractor
Let’s take user input and display the top-matching keywords in real time.  
To make the output neat, we’ll show only the first and last few words of the entered text.


In [9]:
# # Extract top keywords for each content sample
# while True:
#     content = input("Input text to get keywords or press (exit) :")
#     print("\n--------------------------------------------------------------------------------------------------------")
#     if content=="exit" or content=="Exit" :
#         break
#     else:
#         top_keywords = extract_top_keywords(content,keywords,keyword_embeddings,top_n=5)
#         print("Your Content----> ", content[:40], "*********************", content[-40:],"\n")
#         print("Top Keywords---->",top_keywords,"\n")

In [10]:
def extract_top_keywords(content, keywords, model, top_n=5):
    # Encode keywords & content
    keyword_embeddings = model.encode(keywords, convert_to_tensor=True)
    content_embedding = model.encode(content, convert_to_tensor=True)

    # Cosine similarity
    similarities = util.cos_sim(content_embedding, keyword_embeddings).squeeze()

    # Top N
    top_indices = torch.topk(similarities, top_n).indices

    # Convert tensor index → int
    top_keywords = [keywords[i.item()] for i in top_indices]

    return top_keywords


In [11]:
while True:
    content = input("Input text to get keywords or press (exit): ")
    print("\n" + "-"*100)

    if content.lower() == "exit":
        break
    else:
        top_keywords = extract_top_keywords(
            content,
            keywords,
            model,        # ✅ pass model, NOT keyword_embeddings
            top_n=5
        )

        print("Your Content ----> ",
              content[:40], "*********************", content[-40:], "\n")
        print("Top Keywords ---->", top_keywords, "\n")


Input text to get keywords or press (exit): John Doe Email: johndoe@email.com | Phone: 9876543210  Professional Summary: Aspiring data scientist with strong interest in artificial intelligence, machine learning,  and data analytics. Experienced in Python programming, SQL databases, and data visualization. Passionate about applying technology to solve real-world problems.  Education: Bachelor of Computer Applications (BCA) XYZ University Relevant coursework includes software engineering, database management systems,  statistics, and data science.  Technical Skills: Python, Machine Learning, Artificial Intelligence, SQL, Data Science, NLP, Cloud Computing, Cybersecurity, Software Engineering  Projects: •

----------------------------------------------------------------------------------------------------
Your Content ---->  John Doe Email: johndoe@email.com | Phon ********************* urity, Software Engineering  Projects: • 

Top Keywords ----> ['software engineering', 'cybersecurity',